In [3]:
import spacy
from spacy import displacy
from spacy_conll import ConllFormatter
import os
import re
import xlsxwriter

In [30]:
nlp = spacy.load("en_core_web_trf")

In [31]:
#conllformatter = ConllFormatter(nlp, "name") #deleted by Petya
nlp.add_pipe('conll_formatter', after='parser') #changed from conllformatter to 'conll_formatter' by Petya

In [32]:
doc = nlp('And for this "Study in language variation : stress in Russian nouns" course paper, motivation comes from a changeable of language, which helps us to know how language modifies as time goes by.')
conll = doc._.conll_str
print(conll)

1	And			CC	_	19	cc	_	_
2	for			IN	_	19	prep	_	_
3	this			DT	_	16	det	_	_
4	"			``	_	16	punct	_	_
5	Study			VB	_	16	nmod	_	_
6	in			IN	_	5	prep	_	_
7	language			NN	_	8	compound	_	_
8	variation			NN	_	6	pobj	_	_
9	:			:	_	5	punct	_	_
10	stress			NN	_	5	appos	_	_
11	in			IN	_	10	prep	_	_
12	Russian			JJ	_	13	amod	_	_
13	nouns			NNS	_	11	pobj	_	_
14	"			''	_	16	punct	_	_
15	course			NN	_	16	compound	_	_
16	paper			NN	_	2	pobj	_	_
17	,			,	_	19	punct	_	_
18	motivation			NN	_	19	nsubj	_	_
19	comes			VBZ	_	0	ROOT	_	_
20	from			IN	_	19	prep	_	_
21	a			DT	_	22	det	_	_
22	changeable			JJ	_	20	pobj	_	_
23	of			IN	_	22	prep	_	_
24	language			NN	_	23	pobj	_	_
25	,			,	_	22	punct	_	_
26	which			WDT	_	27	nsubj	_	_
27	helps			VBZ	_	22	relcl	_	_
28	us			PRP	_	30	nsubj	_	_
29	to			TO	_	30	aux	_	_
30	know			VB	_	27	ccomp	_	_
31	how			WRB	_	33	advmod	_	_
32	language			NN	_	33	nsubj	_	_
33	modifies			VBZ	_	30	ccomp	_	_
34	as			IN	_	36	mark	_	_
35	time			NN	_	36	nsubj	_	_
36	goes			VBZ	_	33	advcl	_	_
37	by	

In [33]:
def is_int(string):
    try:
        int(string)
        return True
    except Exception:
        return False

In [34]:
def open_file(filename):
    with open(filename, encoding='utf-8') as f:
        text = f.read()
    return text

In [35]:
def find_bad(words):
    erroneus = []
    bad_things = [
        ('NN', 'VBP'),
        ('NNS', 'VBZ'),
    ]
    text = []
    for key, value in words.items():
        text.append(value['text'])
    sent = ' '.join(text)
    pos_subj = ''
    pos_verb = ''
    predicates = []
    for number, char in words.items():
        if char['tag'] == 'ROOT':
            verb = char['text']
            pos_verb = char['pos']
            n = number
            predicates.append([verb, pos_verb, n])
    for number, char in words.items():
        if (char['tag'] == 'conj' or char['tag'] == 'ccomp') and char['s_pos'] == 'VERB':
            verb = char['text']
            pos_verb = char['pos']
            n = number
            predicates.append([verb, pos_verb, n])
    for number, char in words.items():
        if char['tag'] == 'acl':
            verb = char['text']
            pos_verb = char['pos']
            n = number
            predicates.append([verb, pos_verb, n])
    for pred in predicates:
        verb = pred[0]
        if verb.lower() in [
            'bet',
            'bid',
            'broadcast',
            'burst',
            'bust',
            'cast',
            'cost',
            'cut',
            'fit',
            'forecast',
            'hit',
            'hurt',
            'input',
            'knit',
            'let',
            'miswed',
            'preset',
            'put',
            'quit',
            'read',
            'rid',
            'set',
            'shed',
            'shut',
            'slit',
            'split',
            'spread',
            'sublet',
            'sweat',
            'thrust',
            'upset',
            'wed',
            'wet'
        ]:
            break
        pos_verb = pred[1]
        n = pred[2]
        for number, char in words.items():
            if char['dep'] == str(n) and char['tag'] == 'aux':
                pos_verb = char['pos']
                verb = char['text']
        sub = False
        countern = 0
        counterc = 0
        countera = 0
        for number, char in words.items():
            if char['dep'] == str(n) and (char['tag'] == 'nsubj' or char['tag'] == 'csubj'):
                subject = char['text']
                pos_subj = char['pos']
                if pos_subj == 'JJ':
                    return False
                k = number
                sub = True
                if char['tag'] == 'nsubj':
                    countern += 1
                elif char['tag'] == 'csubj':
                    counterc += 1
        if not sub:
            for number, char in words.items():
                if char['dep'] == str(n) and char['tag'] == 'attr':
                    for num, ch in words.items():
                        if ch['dep'] == str(n) and ch['tag'] == 'expl':
                            #print(sent)
                            sub = True
                            subject = char['text']
                            pos_subj = char['pos']
                            k = number
                            countera += 1
        if sub:
            if subject.lower() in [
                'bison',
                'cod',
                'deer',
                'fish',
                'moose',
                'offspring',
                'pike',
                'salmon',
                'sheep',
                'shrimp',
                'swine',
                'trout',
                'boar',
                'buffalo',
                'grouse',
                'elk',
                'fruit',
                'reindeer'
            ]:
                break
            if subject.lower() in 'government data statistics police team'.split():
                pos_subj = 'COLL'
            
            if subject in 'lot majority plenty part most group type class team half percent cent %'.split():
                for number, char in words.items():
                        if char['dep'] == str(k) and char['tag'] == 'prep':
                            if char['text'] == 'of':
                                order = number
                                for number, char in words.items():
                                    if char['dep'] == str(order) and char['tag'] == 'pobj':
                                        pos_subj = char['pos']
                                                
            if subject == 'number' or subject == 'lot':
                for number, char in words.items():
                    if char['dep'] == str(k) and char['tag'] == 'det':
                        if char['text'].lower() == 'a':
                            pos_subj = 'NNS'

            if subject in 'I i we We you You They they'.split():
                pos_subj = 'NNS'
            elif subject in 'She she He he it It'.split():
                pos_subj = 'NN'

            for number, char in words.items():
                if char['dep'] == str(k) and char['tag'] == 'cc':
                    if char['text'].lower() == 'or':
                        for ka, ch in words.items():
                            if ch['dep'] == str(k) and ch['tag'] == 'conj':
                                subject = ch['text']
                                pos_subj = ch['pos']
                                if ch['text'] in 'I i we We you You They they'.split():
                                    pos_subj = 'NNS'
                                elif ch['text'] in 'She she He he it It'.split():
                                    pos_subj = 'NN'
                    else:
                        pos_subj = 'NNS'
                if char['dep'] == str(k) and char['tag'] == 'conj':
                    pos_subj = 'NNS'
            if countern > 1 or counterc > 1 or countera > 1:
                pos_subj = 'NNS'
            pattern = r'[\'"].*' + subject + r'.*[\'"].*' + verb
            if re.search(pattern, sent):
                pos_subj = 'NN'
                #print(sent)
            if (pos_subj, pos_verb) in bad_things:
                return pos_subj, pos_verb, subject, verb
    return False

In [36]:
def search(directory):  #'C:/Users/anton/Рабочий стол/programming/awarl/new/' deleted
    LINES_PER_FILE = 1e9 #line added
    errors = []
    folders = os.listdir(directory)
    folders.sort(key=lambda folder : int(folder))
    for folder in folders:
        print(folder)
        files = os.listdir(directory + folder)
        for file in files:
            text = open_file(directory + folder + '/' + file)
            sentences = text.split('\n') #changed from '@' to '\n'
            counter = 1 #line added
            for sent in sentences:
                if sent == '': #line added
                    continue   #line added
                sent = re.sub(r'<.*>', '', sent)
                sent = re.sub(r' +', r' ', sent)
                sent = sent.replace('+', '')
                sent = sent.replace('*', '')
                #if 'there is' in sent.lower():
                    #print(sent)
                doc = nlp(sent)
                conll = doc._.conll_str
                lines = conll.splitlines()
                words = {}
                for line in lines:
                    word = line.split('\t')
                    if len(word) == 10:
                        # print(word)
                        words[int(word[0])] = {'text': word[1], 's_pos': word[3], 'pos': word[4], 'dep': word[6], 'tag': word[7]}
                try:
                    a = find_bad(words)
                    if a:
                        errors.append([
                            sent,
                            f'{a[2]} {a[3]}',
                            f'{a[0]}, {a[1]}',
                            file,
                            folder
                        ])
                except Exception:
                    print(sent)
                counter += 1                  #line added
                if counter == LINES_PER_FILE: #line added
                    break                     #line added
    return errors

In [37]:
def writeln(errors, filename):
    workbook = xlsxwriter.Workbook(filename)
    worksheet = workbook.add_worksheet()
    row = 0
    col = 0
    for error in errors:
        for i in range(5):
            worksheet.write(row, col + i, error[i])
        row += 1
    workbook.close()

In [38]:
%%time

a = search('test/evaluate/medium/')
writeln(a, 'subj_verb_new.xlsx')

1
CPU times: user 20 s, sys: 6.55 ms, total: 20 s
Wall time: 5.2 s
